In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# import zipfile
# import os

# zip_path = "/content/drive/MyDrive/DEEP LEARNING PROJECTS(VIHARATECH)/food_cassify_data.zip"  # Update this with your file path
# extract_path = "/content/drive/MyDrive/DEEP LEARNING PROJECTS(VIHARATECH)/ food_classification_data1"  # Choose where to extract

# # Create directory if it does not exist
# os.makedirs(extract_path, exist_ok=True)

# # Extract the ZIP file
# with zipfile.ZipFile(zip_path, 'r') as zip_ref:
#     zip_ref.extractall(extract_path)

# print("Extraction completed!")

In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import os
import sys
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D,MaxPool2D,Flatten
from tensorflow.keras.activations import relu,sigmoid
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
import warnings
warnings.filterwarnings('ignore')
class CUSTOM_MODEL:
  def __init__(self,training_data,testing_data,validation_data):
    try:
      #imageprocessing for all the data
      self.train_formats = ImageDataGenerator(rescale = 1.0/255,
                                   shear_range=0.2,
                                   zoom_range=0.3,
                                   horizontal_flip = True)

      self.labels=os.listdir(training_data)
      self.train_data = self.train_formats.flow_from_directory(training_data,
                                  target_size=(256,256),
                                  classes=self.labels,
                                  class_mode='categorical',
                                  batch_size=10)
      self.valid_data = self.train_formats.flow_from_directory(validation_data,
                                  target_size=(256,256),
                                  classes=self.labels,
                                  class_mode='categorical',
                                  batch_size=10)
      self.test_data = self.train_formats.flow_from_directory(testing_data,
                                  target_size=(256,256),
                                  classes=self.labels,
                                  class_mode='categorical',
                                    batch_size=20)

    except Exception as e:
      er_type,er_msg,line_no=sys.exc_info()
      print(f'{er_type} <------>{er_msg}<-------->{line_no.tb_lineno}')

  def model_building(self):
    try:
      #model building using custom structure
      self.model = Sequential()

      # first kernel and max poo layers
      self.model.add(Conv2D(128 , kernel_size = (3,3) , input_shape = (int(256),int(256),3) , padding='same',activation='relu'))
      self.model.add(MaxPool2D(pool_size = (2,2)))


      # second kernel and max poo layers
      self.model.add(Conv2D(64 , kernel_size = (3,3), padding='same',activation='relu'))
      self.model.add(MaxPool2D(pool_size = (2,2)))


      # third kernel and max poo layers
      self.model.add(Conv2D(32 , kernel_size = (3,3), padding='same',activation='relu'))
      self.model.add(MaxPool2D(pool_size = (2,2)))


      # forth kernel and max poo layers
      self.model.add(Conv2D(6 , kernel_size = (3,3), padding='same',activation='relu'))
      self.model.add(MaxPool2D(pool_size = (2,2)))


      self.model.add(Flatten()) # one dimensional array


      # above 1d data give to ANN
      self. model.add(Dense(128,activation='relu')) # hiddel layer 1
      self.model.add(Dense(64,activation = 'relu')) # hiddel layer 2


      # output layer
      self.model.add(Dense(34,activation='softmax'))

      print(self.model.summary())


    except Exception as e:
      er_type,er_msg,line_no=sys.exc_info()
      print(f'{er_type} <------>{er_msg}<-------->{line_no.tb_lineno}')

  def training_epoch(self):
    try:
      self.model_building()
      self.model.compile(optimizer='adam',metrics=['accuracy'],loss='categorical_crossentropy')
      self.model.fit(self.train_data,
                     epochs=15,validation_data=self.valid_data)

      model_path="/content/drive/MyDrive/DEEP LEARNING PROJECTS(VIHARATECH)/ food_classification_data1/food_cassify_data"
      self.model.save(f'{model_path}/custom_model.h5')


    except Exception as e:
      er_type,er_msg,line_no=sys.exc_info()
      print(f'{er_type} <------>{er_msg}<-------->{line_no.tb_lineno}')


  def model_evaluation(self):
    try:
      output_path="/content/drive/MyDrive/DEEP LEARNING PROJECTS(VIHARATECH)/ food_classification_data1/ custom_outputs"
      model_path="/content/drive/MyDrive/DEEP LEARNING PROJECTS(VIHARATECH)/ food_classification_data1/ custom_outputs/custom_mode.h5"
      self.model_path=model_path
      self.model=load_model(self.model_path)
      self.output_path = os.path.join(output_path, "custom_classification_report.json")
      self.class_names = np.array(list(self.test_data.class_indices.keys()))
      y_true = np.array(self.test_data.classes)
      y_pred_prob = self.model.predict(self.test_data)
      y_pred = np.argmax(y_pred_prob, axis=1)
      conf_matrix = confusion_matrix(y_true, y_pred)

      total_samples = conf_matrix.sum()
      accuracy = (np.sum(np.diag(conf_matrix))) / total_samples

      report = {}

      report["Overall_Accuracy"] = round(accuracy, 4)

      num_classes = len(self.class_names)
      for i in range(num_classes):
          TP = conf_matrix[i, i]
          FP = np.sum(conf_matrix[:, i]) - TP
          FN = np.sum(conf_matrix[i, :]) - TP
          TN = total_samples - (TP + FP + FN + TP)

          precision = TP / (TP + FP) if (TP + FP) > 0 else 0
          recall = TP / (TP + FN) if (TP + FN) > 0 else 0
          f1_score = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

          report[self.class_names[i]] = {
              "TP": int(TP),
              "TN": int(TN),
              "FP": int(FP),
              "FN": int(FN),
              "Accuracy": round((TP + TN) / (TP + TN + FP + FN), 4),
              "Precision": round(precision, 4),
              "Recall": round(recall, 4),
              "F1-Score": round(f1_score, 4)
          }

      import json
      with open(self.output_path,'w') as f:
        json.dump(report,f)
    except Exception as e:
      er_type,er_msg,line_no=sys.exc_info()
      print(f'{er_type} <------>{er_msg}<-------->{line_no.tb_lineno}')




if __name__=="__main__":
  training_data="/content/drive/MyDrive/DEEP LEARNING PROJECTS(VIHARATECH)/ food_classification_data1/food_cassify_data/training_data_150"
  testing_data="/content/drive/MyDrive/DEEP LEARNING PROJECTS(VIHARATECH)/ food_classification_data1/food_cassify_data/testing_data_20"
  validation_data="/content/drive/MyDrive/DEEP LEARNING PROJECTS(VIHARATECH)/ food_classification_data1/food_cassify_data/validation_data_30"
  obj=CUSTOM_MODEL(training_data,testing_data,validation_data)
  obj.model_building()

Found 5094 images belonging to 34 classes.
Found 1020 images belonging to 34 classes.
Found 680 images belonging to 34 classes.


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_16 (Conv2D)                   │ (None, 256, 256, 128)       │           3,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_16 (MaxPooling2D)      │ (None, 128, 128, 128)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_17 (Conv2D)                   │ (None, 128, 128, 64)        │          73,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_17 (MaxPooling2D)      │ (None, 64, 64, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_18 (Conv2D)                   │ (None, 64, 64, 32)          │          18,464 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_18 (MaxPooling2D)      │ (None, 32, 32, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_19 (Conv2D)                   │ (None, 32, 32, 6)           │           1,734 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_19 (MaxPooling2D)      │ (None, 16, 16, 6)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_4 (Flatten)                  │ (None, 1536)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ (None, 128)                 │         196,736 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ (None, 34)                  │           2,210 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 304,776 (1.16 MB)

 Trainable params: 304,776 (1.16 MB)

 Non-trainable params: 0 (0.00 B)

None


In [ ]:
obj.training_epoch()

In [10]:
obj.model_evaluation()

34/34 ━━━━━━━━━━━━━━━━━━━━ 78s 2s/step
